## Task1 ****估测价格 liner regression****
regreesion：piecewise liner
price = f(年款*，公里*，MSRP，自动/手动，省*，颜色**，排气量，月份，来源，地区库存***）
         
feature definition:
*省：省¥/全国¥ *100
**颜色：省¥/全国¥ *100
***颜色：省¥/全国¥ *100           
****地区库存（该车发布日期前改地区的库存）  
            

Variable：
1）公里 M1，M2，M3...； （分段）   y-price，x-miles
2）年龄 A1，A2，A3...； （分段）

Way1:raw data
Way2:categorical
Way3:index/derive

In [7]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 500)   # print all info in cell
pd.set_option('display.max_rows', 500) 
pd.set_option('display.max_colwidth', -1)   # print all info in cell

In [8]:
# display chinese
import matplotlib.font_manager as mfm
import matplotlib.pyplot as plt
font_path = "/Users/wyywenyi/anaconda2/lib/python2.7/site-packages/matplotlib/mpl-data/fonts/ttf/simhei.ttf"
prop = mfm.FontProperties(fname=font_path)

In [9]:
df = pd.read_csv('corolla_2006_2019_v3.csv', encoding = 'utf-8')

In [10]:
# 排量

def liter_num(liter):
    if isinstance(liter,float):
        pass
    else:
        liter = liter[liter.find('.')-1 : liter.find('.')+2]
    
    return liter
    
df['liter_num'] = df.att_liter.apply(liter_num)

df = df[df.liter_num != ''].reset_index()

### 1 - base model
#### X: independent variabe
* model_year
* miles
* manual_or_automated
* liter
* province
* age_month
* color
* litter
* month(publish_month)
* source

#### Y: predict varibable
* final price


In [11]:
df.head(3)

,index,Unnamed: 0,Unnamed: 0.1,att_if_CVT,att_if_GL,att_if_hybrid,att_if_update,att_internal_material,att_liter,att_make,att_model_name,att_trim_1,att_trim_2,id,city,auto_full_name,model_year,source,publish_date,sale_date,date_on_market,final_price,original_price,color,miles,manual_or_automated,plate_date,plate_year,age_month,price_adj,age_year,province,province_eng,liter_num
0,0,0,0,CVT,GLi炫酷版,NaN,NaN,NaN,1.6L,Toyota,Corolla,2016款,NaN,216814907,东莞,丰田 卡罗拉 2016款 1.6L CVT GLi炫酷版,2016,merchandise_source,2019-03-25 08:05:42,2019-03-28 13:03:01,3,8.69,8.69,white,3.00,AT,2016-12-01 00:00:00,2016,27,0.0,2,广东,guangdong,1.6
1,1,1,17,CVT,GL,NaN,NaN,i真皮版,1.6L,Toyota,Corolla,2014款,NaN,216818028,东莞,丰田-卡罗拉 2014款 1.6L CVT GL-i真皮版,2014,platform_source,2019-03-25 09:42:23,2019-03-28 09:43:23,3,8.68,8.68,white,12.09,AT,2016-04-01 00:00:00,2016,35,0.0,2,广东,guangdong,1.6
2,2,2,18,CVT,GL,NaN,NaN,i真皮版,1.6L,Toyota,Corolla,2014款,NaN,216818028,东莞,丰田-卡罗拉 2014款 1.6L CVT GL-i真皮版,2014,platform_source,2019-03-25 09:42:23,2019-03-28 09:43:23,3,8.68,8.68,white,12.09,AT,2016-04-01 00:00:00,2016,35,0.0,2,广东,guangdong,1.6


In [12]:
model_year = pd.get_dummies(df.model_year,drop_first = True)
miles = df.miles
at_mt = pd.get_dummies(df.manual_or_automated,drop_first=True)
liter = df.liter_num
province = pd.get_dummies(df.province_eng,drop_first=True)
age_month = df.age_month
color = pd.get_dummies(df.color,drop_first=True)
month = pd.to_datetime(df.publish_date).dt.month
source = pd.get_dummies(df.source,drop_first=True)

In [13]:
X = pd.concat([model_year,miles,at_mt,province,age_month,color,month,source],axis = 1)

In [14]:
y = df.final_price

In [15]:
feature_cols = X.columns

In [16]:
feature_cols

Index([                 2007,                  2008,                  2009,
                        2010,                  2011,                  2012,
                        2013,                  2014,                  2015,
                        2016,                  2017,                  2018,
                        2019,              u'miles',                 u'MT',
                  u'beijing',          u'chongqing',             u'fujian',
                    u'gansu',          u'guangdong',            u'guangxi',
                  u'guizhou',             u'hainan',              u'hebei',
               u'heilongjia',              u'henan',              u'hubei',
                    u'hunan',            u'jiangsu',            u'jiangxi',
                    u'jilin',           u'liaoning',          u'neimenggu',
                  u'ningxia',            u'qinghai',           u'shandong',
                 u'shanghai',            u'shannxi',             u'shanxi',
            

In [17]:
# Splitting X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [18]:
# import model
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [19]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

10.146585593413338
[ 0.50128264  0.74003134  0.54204132  0.05600653  0.3780594   0.35803387
  0.28610873  0.34846438  2.38395071  1.21294019  0.81536961  1.71629277
  1.97154963 -0.02040773 -1.1975246   0.21386463  0.12181843  0.40088554
  0.1990049   0.51315826  0.66190999  0.05150029  0.49329075  0.46311798
  0.3231475   0.12501997  0.07789479  0.21021158  0.25113594  0.0826484
  0.36748886  0.44869032  0.41459606  0.37775114  0.59735314  0.04525613
  0.03645489  0.3468594   0.3515065   0.01787497  0.5414029   0.12761439
  0.62265656  0.29828375  0.27165387 -0.05092294  0.02455556  0.07769699
  0.03917695  0.08890964 -0.00630231  0.13072412  0.04384816]


In [20]:
# pair the feature names with the coefficients
list(zip(feature_cols, linreg.coef_))

[(2007, 0.5012826444947542),
 (2008, 0.7400313444852905),
 (2009, 0.5420413155642927),
 (2010, 0.05600653232148595),
 (2011, 0.37805939531647337),
 (2012, 0.3580338672499973),
 (2013, 0.28610873421850025),
 (2014, 0.3484643771842232),
 (2015, 2.383950705633821),
 (2016, 1.2129401851479158),
 (2017, 0.8153696076268873),
 (2018, 1.71629276553299),
 (2019, 1.9715496319285681),
 (u'miles', -0.020407725640703367),
 (u'MT', -1.197524601342606),
 (u'beijing', 0.2138646256912608),
 (u'chongqing', 0.12181842557381113),
 (u'fujian', 0.40088553637011926),
 (u'gansu', 0.19900489537741187),
 (u'guangdong', 0.5131582649767584),
 (u'guangxi', 0.6619099902878746),
 (u'guizhou', 0.05150028860764477),
 (u'hainan', 0.49329074774849196),
 (u'hebei', 0.4631179791837218),
 (u'heilongjia', 0.3231475028091005),
 (u'henan', 0.12501997000850695),
 (u'hubei', 0.07789479192969884),
 (u'hunan', 0.21021158103549925),
 (u'jiangsu', 0.25113594054616184),
 (u'jiangxi', 0.08264839776406259),
 (u'jilin', 0.3674888600252

In [21]:
# make predictions on the testing set
y_pred = linreg.predict(X_test)

In [22]:
from sklearn import metrics
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.8704287318198483


In [27]:
X_test_1 = X_test.iloc[1,]

In [31]:
linreg.predict([X_test_1])

array([8.16164604])

In [32]:
X_test_1

2007                  0.00 
2008                  0.00 
2009                  0.00 
2010                  0.00 
2011                  0.00 
2012                  0.00 
2013                  0.00 
2014                  1.00 
2015                  0.00 
2016                  0.00 
2017                  0.00 
2018                  0.00 
2019                  0.00 
miles                 5.92 
MT                    0.00 
beijing               0.00 
chongqing             0.00 
fujian                0.00 
gansu                 0.00 
guangdong             0.00 
guangxi               0.00 
guizhou               0.00 
hainan                0.00 
hebei                 0.00 
heilongjia            0.00 
henan                 0.00 
hubei                 0.00 
hunan                 0.00 
jiangsu               0.00 
jiangxi               0.00 
jilin                 0.00 
liaoning              0.00 
neimenggu             0.00 
ningxia               0.00 
qinghai               0.00 
shandong            

In [38]:
aa = [0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,17,0,0,0,1,1,1,0]

In [39]:
linreg.predict([aa])

array([10.72321443])